In [12]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.callbacks import TensorBoard
from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose,Activation,add,multiply
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras import regularizers
from keras.layers.merge import concatenate
from keras.layers.core import Lambda
import tensorflow as tf
from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam

In [13]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
im_width = 224
im_height = 224
border = 5
path_train = '/media/poudelas/24f81228-8af6-4609-aa17-d48ef8ddc61f/Data/CVC-ClinicDB/train/'
path_test = '/media/poudelas/24f81228-8af6-4609-aa17-d48ef8ddc61f/Data/CVC-ClinicDB/validation/'

In [14]:
# Get and resize train images and masks
def get_data(path, train=True):
    ids = next(os.walk(path + "image"))[2]
    X = np.zeros((len(ids), 224, 224, 3), dtype=np.uint8)
    y = np.zeros((len(ids), 224, 224, 1), dtype=np.bool)

    print('Getting and resizing images ... ')
    for n, id_ in tqdm_notebook(enumerate(ids), total=len(ids)):
        # Load images
        # img = load_img(path + '/image/' + id_, grayscale=False)
        img = imread(path + '/image/' + id_)[:, :, :3]
        img = resize(img, (224, 224), mode='constant', preserve_range=True)
        X[n] = img
#         id_ = list(id_)
#         id_[3] = 'y'
#         string = ''.join(id_)
        mask = img_to_array(load_img(path + 'mask/' + id_, grayscale=True))
        mask = resize(mask, (224, 224, 1), mode='constant', preserve_range=True)

        X[n] = img
        y[n] = mask
    return X, y

In [15]:
X_train, Y_train = get_data(path_train, train=True)
X_valid, Y_valid = get_data(path_test,train=True)

Getting and resizing images ... 


/home/poudelas/anaconda3/envs/auto/lib/python3.6/site-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/home/poudelas/anaconda3/envs/auto/lib/python3.6/site-packages/keras_preprocessing/image/utils.py:104: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '



Getting and resizing images ... 


In [16]:
#X_train, Y_train = get_data(path_train, train=True)
BATCH_SIZE = 10
seed = 1
from keras.preprocessing import image
# Creating the training Image and Mask generator
image_datagen = image.ImageDataGenerator(shear_range=0.5, rotation_range=50, zoom_range=0.2, width_shift_range=0.2, height_shift_range=0.2, fill_mode='reflect')
mask_datagen = image.ImageDataGenerator(shear_range=0.5, rotation_range=50, zoom_range=0.2, width_shift_range=0.2, height_shift_range=0.2, fill_mode='reflect')

# Keep the same seed for image and mask generators so they fit together

image_datagen.fit(X_train, augment=True, seed=seed)
mask_datagen.fit(Y_train, augment=True, seed=seed)

x=image_datagen.flow(X_train,batch_size=BATCH_SIZE,shuffle=True, seed=seed)
y=mask_datagen.flow(Y_train,batch_size=BATCH_SIZE,shuffle=True, seed=seed)

In [17]:
#X_train, Y_train = get_data(path_train, train=True)
BATCH_SIZE = 10
seed = 1
from keras.preprocessing import image
# Creating the training Image and Mask generator
image_datagen = image.ImageDataGenerator()
mask_datagen = image.ImageDataGenerator()

# Keep the same seed for image and mask generators so they fit together

image_datagen.fit(X_valid, seed=seed)
mask_datagen.fit(Y_valid, seed=seed)

x_val=image_datagen.flow(X_valid,batch_size=BATCH_SIZE,seed=seed)
y_val=mask_datagen.flow(Y_valid,batch_size=BATCH_SIZE,seed=seed)

In [18]:
train_generator = zip(x, y)
val_generator = zip(x_val, y_val)
smooth = 1

In [21]:
def dice_coef(y_true, y_pred):# y_true为真实准确值，y_pred为预测值
    y_true_f = K.flatten(y_true)# 捋直
    y_pred_f = K.flatten(y_pred)# 捋直
    # K.sum不加axi（指定方向求和，返回对应方向向量）,则为全元素求和，返回一个数字
    intersection = K.sum(y_true_f * y_pred_f)# 求预测准确的结果（真实准确值和预测值的交集）
    # 原始公式：（2*预测准确值）/（真实准确值+预测值），越大效果越好
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

def jaccard_distance_loss(y_true, y_pred, smooth=100):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return (1 - jac) * smooth
def IoULoss(y_true, y_pred, smooth=1e-6):
    
    #flatten label and prediction tensors
    inputs = K.flatten(inputs)
    targets = K.flatten(targets)
    
    intersection = K.sum(K.dot(targets, inputs))
    total = K.sum(targets) + K.sum(inputs)
    union = total - intersection
    
    IoU = (intersection + smooth) / (union + smooth)
    return 1 - IoU

In [38]:
from load_model import *
model = proposed_method()
print(model.summary())

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, None, None, 3 864         input_2[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, None, None, 3 128         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, None, None, 3 0           stem_bn[0][0]                    
____________________________________________________________________________________________

In [39]:
from keras.optimizers import Adam
model.compile(optimizer=Adam(lr = 0.001),loss='binary_crossentropy', metrics=[dice_coef,mean_iou,jaccard_distance_loss])
# Fit model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
tensorboard = TensorBoard(log_dir='./proposed_Unet',histogram_freq=0,write_graph=True,write_images=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                   patience=4, min_lr=0.00001, verbose=1,cooldown=1)
earlystopper = EarlyStopping(patience=10, verbose=1)
checkpointer = ModelCheckpoint('proposed_Unet.h5', verbose=1, save_best_only=True)
results = model.fit_generator(train_generator, validation_data=val_generator, validation_steps=10, steps_per_epoch=250,
                              epochs=100, callbacks=[tensorboard, checkpointer,reduce_lr,earlystopper])

Epoch 1/100
250/250 [==============================] - 85s 342ms/step - loss: 0.1601 - dice_coef: 0.5904 - mean_iou: 0.6693 - jaccard_distance_loss: 0.1170 - val_loss: 0.1013 - val_dice_coef: 0.7434 - val_mean_iou: 0.7547 - val_jaccard_distance_loss: 0.0581

Epoch 00001: val_loss improved from inf to 0.10128, saving model to proposed_Unet.h5
Epoch 2/100
250/250 [==============================] - 69s 276ms/step - loss: 0.0820 - dice_coef: 0.7715 - mean_iou: 0.7788 - jaccard_distance_loss: 0.0501 - val_loss: 0.0686 - val_dice_coef: 0.8320 - val_mean_iou: 0.7970 - val_jaccard_distance_loss: 0.0363

Epoch 00002: val_loss improved from 0.10128 to 0.06860, saving model to proposed_Unet.h5
Epoch 3/100
250/250 [==============================] - 69s 275ms/step - loss: 0.0539 - dice_coef: 0.8449 - mean_iou: 0.8134 - jaccard_distance_loss: 0.0324 - val_loss: 0.0414 - val_dice_coef: 0.8430 - val_mean_iou: 0.8259 - val_jaccard_distance_loss: 0.0288

Epoch 00003: val_loss improved from 0.06860 to 0.


Epoch 00025: val_loss did not improve from 0.02218
Epoch 26/100
250/250 [==============================] - 73s 293ms/step - loss: 0.0219 - dice_coef: 0.9354 - mean_iou: 0.9166 - jaccard_distance_loss: 0.0133 - val_loss: 0.0248 - val_dice_coef: 0.9321 - val_mean_iou: 0.9172 - val_jaccard_distance_loss: 0.0129

Epoch 00026: val_loss did not improve from 0.02218
Epoch 00026: early stopping


In [40]:
path_valid = '/media/poudelas/24f81228-8af6-4609-aa17-d48ef8ddc61f/Data/ETIS-LaribPolypDB/'

In [41]:
# Get and resize train images and masks
def get_data(path, train=True):
    ids = next(os.walk(path + "image"))[2]
    X = np.zeros((len(ids), 224, 224, 3), dtype=np.uint8)
    y = np.zeros((len(ids), 224, 224, 1), dtype=np.bool)

    print('Getting and resizing images ... ')
    for n, id_ in tqdm_notebook(enumerate(ids), total=len(ids)):
        # Load images
        # img = load_img(path + '/image/' + id_, grayscale=False)
        img = imread(path + '/image/' + id_)[:, :, :3]
        img = resize(img, (224, 224), mode='constant', preserve_range=True)
        X[n] = img
#         id_ = list(id_)
#         id_[3] = 'y'
#         string = ''.join(id_)
        mask = img_to_array(load_img(path + 'mask/' + 'p'+id_, grayscale=True))
        mask = resize(mask, (224, 224, 1), mode='constant', preserve_range=True)

        X[n] = img
        y[n] = mask
    return X, y

In [42]:
X_test, Y_test = get_data(path_valid,train=True)

Getting and resizing images ... 


/home/poudelas/anaconda3/envs/auto/lib/python3.6/site-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/home/poudelas/anaconda3/envs/auto/lib/python3.6/site-packages/keras_preprocessing/image/utils.py:104: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [11]:
from keras.models import load_model
model = load_model('Efficient_Unet.h5')

ValueError: Unknown activation function:swish

In [25]:
# preds_val = model.evaluate(X_test,Y_test, verbose=0)
# print(preds_val[0])
# print(preds_val[1])
# print(preds_val[2])
# print(preds_val[3])

0.3197446383383809
0.47146571169094165
0.9247918481729469
0.09346010178631665


In [35]:
preds_val = model.evaluate(X_test,Y_test, verbose=1)
print(preds_val[0])
print(preds_val[1])
print(preds_val[2])
print(preds_val[3])

196/196 [==============================] - 1s 7ms/step
0.19511553097744377
0.487867005017339
0.9159586222804322
0.08460415489211374


In [43]:
preds_val = model.evaluate(X_test,Y_test, verbose=1)
print(preds_val[0])
print(preds_val[1])
print(preds_val[2])
print(preds_val[3])

196/196 [==============================] - 1s 8ms/step
0.1245488068278955
0.6221618031968876
0.9170576917881869
0.041947733686894785


In [36]:
preds_train = model.predict(X_train, verbose=1)
preds_val = model.predict(X_valid, verbose=1)
preds_test = model.predict(X_test,verbose=1)

# Threshold predictions
preds_train_t = (preds_train > 0.5).astype(np.uint8)
preds_val_t = (preds_val > 0.5).astype(np.uint8)
preds_test_t = (preds_test>0.5).astype(np.uint8)

196/196 [==============================] - 1s 6ms/step


In [37]:
import cv2
for i in range(len(preds_test_t)):
    #cv2.imwrite('./visualize/Unet_plus_plus/image/original'+str(i)+'.jpg',cv2.cvtColor(X_train[i], cv2.COLOR_BGR2RGB))
    cv2.imwrite('/media/poudelas/24f81228-8af6-4609-aa17-d48ef8ddc61f/Data/ETIS-LaribPolypDB/predicted_mask/p'+str(i+1)+'.tif',np.squeeze(preds_test_t[i]*255))